# OBRIGATORIA


In [1]:
%pip install pandas
%pip install matplot
%pip install mlxtend

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules, apriori
from importlib import reload
import numpy as np
import csv

plt=reload(plt)

# FILE


In [25]:
PROJECT = "auto"
DF_COMMENTS_FILE_NAME = "../csv/comments-" + PROJECT +".csv"
FILE_NAME_TABELA_AGRUPADA = "../" + PROJECT +"/tabela_agrupada.csv"
FILE_NAME_TABELA_MERGE = "../" + PROJECT +"/tabela_merge.csv"
FILE_NAME_TABELA_DIFERENCA_COMENTARIOS = "../" + PROJECT +"/tabela_diferenca_comentarios.csv"
FILE_NAME_TABELA_DIFERENCA_COMENTARIOS_BINARIO = "../" + PROJECT +"/tabela_diferenca_comentarios_binario.csv"
FILE_NAME_REFACTORING = "../csv/refactorings-" + PROJECT +".csv"
FILE_NAME_REFACTORING_AGRUPADO = "../" + PROJECT +"/refactoring_agrupado.csv"
FILE_NAME_TABELA_COMPLETA = "../" + PROJECT +"/tabela_completa.csv"
FILE_NAME_TABELA_REGRAS = "../" + PROJECT +"/tabela_regras.csv"

# TRABALHANDO NOS CALCULOS

In [26]:
DF_COMMENTS = pd.read_csv(DF_COMMENTS_FILE_NAME)
DF_COMMENTS.drop(columns=['12_HASH_CLASSPATH', '13_PARENTHASH_CLASSPATH'], inplace=True)

In [27]:
# Agrupamos pelo '11_HASH_CLASSPATH' e '12_PARENTHASH_CLASSPATH' e somamos as outras colunas
agrupado = DF_COMMENTS.groupby(["10_HASH", "11_PARENTSHASH"]).sum().reset_index()

# Renomeando as colunas para os novos nomes
agrupado.rename(columns={
    "16_SEGMENTOS": "QNT_SEGMENTOS",
    "18_ISBLOCKCOMMENT": "QNT_ISBLOCKCOMMENT",
    "19_ISLINECOMMENT": "QTN_ISLINECOMMENT",
    "20_ISJAVADOCCOMMENT": "QTN_ISJAVADOCCOMMENT"
}, inplace=True) 


agrupado.to_csv(FILE_NAME_TABELA_AGRUPADA, index=False)

In [28]:
# Juntando o DataFrame consigo mesmo para calcular as diferenças

merged_df = pd.merge(agrupado, agrupado, right_on="10_HASH", left_on="11_PARENTSHASH", suffixes=('', '_PARENTE'))

merged_df.drop(columns=['10_HASH_PARENTE', '11_PARENTSHASH_PARENTE'], inplace=True)

merged_df.to_csv(FILE_NAME_TABELA_MERGE, index=False)

# CALCULO DIFERENCAS

In [29]:

# Calculando as diferenças
merged_df["DIFERENCA_QNT_ISBLOCKCOMMENT"] = merged_df["QNT_ISBLOCKCOMMENT"] - merged_df["QNT_ISBLOCKCOMMENT_PARENTE"]
merged_df["DIFERENCA_QTN_ISLINECOMMENT"] = merged_df["QTN_ISLINECOMMENT"] - merged_df["QTN_ISLINECOMMENT_PARENTE"]
merged_df["DIFERENCA_QTN_ISJAVADOCCOMMENT"] = merged_df["QTN_ISJAVADOCCOMMENT"] - merged_df["QTN_ISJAVADOCCOMMENT_PARENTE"]
merged_df["DIFERENCA_QNT_SEGMENTOS"] = merged_df["QNT_SEGMENTOS"] - merged_df["QNT_SEGMENTOS_PARENTE"]

merged_df.drop(columns=['QNT_ISBLOCKCOMMENT',  'QTN_ISLINECOMMENT', 'QTN_ISJAVADOCCOMMENT', 'QNT_SEGMENTOS',
               'QNT_ISBLOCKCOMMENT_PARENTE', 'QTN_ISLINECOMMENT_PARENTE', 'QTN_ISJAVADOCCOMMENT_PARENTE', 'QNT_SEGMENTOS_PARENTE'], inplace=True)

merged_df.to_csv(FILE_NAME_TABELA_DIFERENCA_COMENTARIOS, index=False)
df = merged_df.copy()

In [30]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE'] = None
df['DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou'] = None
df['DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu'] = None
df['DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QNT_ISBLOCKCOMMENT'] > 0, 'DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou'] = 1
df.loc[df['DIFERENCA_QNT_ISBLOCKCOMMENT'] < 0, 'DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu'] = 1
df.loc[df['DIFERENCA_QNT_ISBLOCKCOMMENT'] == 0, 'DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QNT_ISBLOCKCOMMENT'] > 0, 'DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE'] = df['DIFERENCA_QNT_ISBLOCKCOMMENT']
df.loc[df['DIFERENCA_QNT_ISBLOCKCOMMENT'] < 0, 'DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE'] = df['DIFERENCA_QNT_ISBLOCKCOMMENT']
df.loc[df['DIFERENCA_QNT_ISBLOCKCOMMENT'] == 0, 'DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE'] = 0

df['DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve'].fillna(0, inplace=True)

df.drop(columns=['DIFERENCA_QNT_ISBLOCKCOMMENT', 'DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE'], inplace=True)

In [31]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QTN_ISLINECOMMENT_CHANGE'] = None
df['DIFERENCA_QTN_ISLINECOMMENT_Aumentou'] = None
df['DIFERENCA_QTN_ISLINECOMMENT_Diminuiu'] = None
df['DIFERENCA_QTN_ISLINECOMMENT_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QTN_ISLINECOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_ISLINECOMMENT'] > 0, 'DIFERENCA_QTN_ISLINECOMMENT_Aumentou'] = 1
df.loc[df['DIFERENCA_QTN_ISLINECOMMENT'] < 0, 'DIFERENCA_QTN_ISLINECOMMENT_Diminuiu'] = 1
df.loc[df['DIFERENCA_QTN_ISLINECOMMENT'] == 0, 'DIFERENCA_QTN_ISLINECOMMENT_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QTN_ISLINECOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_ISLINECOMMENT'] > 0, 'DIFERENCA_QTN_ISLINECOMMENT_CHANGE'] = df['DIFERENCA_QTN_ISLINECOMMENT']
df.loc[df['DIFERENCA_QTN_ISLINECOMMENT'] < 0, 'DIFERENCA_QTN_ISLINECOMMENT_CHANGE'] = df['DIFERENCA_QTN_ISLINECOMMENT']
df.loc[df['DIFERENCA_QTN_ISLINECOMMENT'] == 0, 'DIFERENCA_QTN_ISLINECOMMENT_CHANGE'] = 0

df['DIFERENCA_QTN_ISLINECOMMENT_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISLINECOMMENT_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISLINECOMMENT_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISLINECOMMENT_Manteve'].fillna(0, inplace=True)

df.drop(columns=['DIFERENCA_QTN_ISLINECOMMENT', 'DIFERENCA_QTN_ISLINECOMMENT_CHANGE'], inplace=True)

In [32]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QNT_SEGMENTOS_CHANGE'] = None
df['DIFERENCA_QNT_SEGMENTOS_Aumentou'] = None
df['DIFERENCA_QNT_SEGMENTOS_Diminuiu'] = None
df['DIFERENCA_QNT_SEGMENTOS_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QNT_SEGMENTOS_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QNT_SEGMENTOS'] > 0, 'DIFERENCA_QNT_SEGMENTOS_Aumentou'] = 1
df.loc[df['DIFERENCA_QNT_SEGMENTOS'] < 0, 'DIFERENCA_QNT_SEGMENTOS_Diminuiu'] = 1
df.loc[df['DIFERENCA_QNT_SEGMENTOS'] == 0, 'DIFERENCA_QNT_SEGMENTOS_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QNT_SEGMENTOS_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QNT_SEGMENTOS'] > 0, 'DIFERENCA_QNT_SEGMENTOS_CHANGE'] = df['DIFERENCA_QNT_SEGMENTOS']
df.loc[df['DIFERENCA_QNT_SEGMENTOS'] < 0, 'DIFERENCA_QNT_SEGMENTOS_CHANGE'] = df['DIFERENCA_QNT_SEGMENTOS']
df.loc[df['DIFERENCA_QNT_SEGMENTOS'] == 0, 'DIFERENCA_QNT_SEGMENTOS_CHANGE'] = 0

df['DIFERENCA_QNT_SEGMENTOS_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QNT_SEGMENTOS_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QNT_SEGMENTOS_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QNT_SEGMENTOS_Manteve'].fillna(0, inplace=True)

df.drop(columns=['DIFERENCA_QNT_SEGMENTOS', 'DIFERENCA_QNT_SEGMENTOS_CHANGE'], inplace=True)

In [33]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'] = None
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou'] = None
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu'] = None
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] > 0, 'DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou'] = 1
df.loc[df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] < 0, 'DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu'] = 1
df.loc[df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] == 0, 'DIFERENCA_QTN_ISJAVADOCCOMMENT_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] > 0, 'DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'] = df['DIFERENCA_QTN_ISJAVADOCCOMMENT']
df.loc[df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] < 0, 'DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'] = df['DIFERENCA_QTN_ISJAVADOCCOMMENT']
df.loc[df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] == 0, 'DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'] = 0

df['DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Manteve'].fillna(0, inplace=True)

df.drop(columns=['DIFERENCA_QTN_ISJAVADOCCOMMENT', 'DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'], inplace=True)
df.to_csv(FILE_NAME_TABELA_DIFERENCA_COMENTARIOS_BINARIO, index=False)

# Refactorings

In [34]:
DF_REFACTORING = pd.read_csv(FILE_NAME_REFACTORING)
DF_REFACTORING.drop(columns=['11_HASH_CLASSPATH', '13_REFACTORINGTYPE'], inplace=True)

# PARA CADA 10_HASH QUANTO QUANTAS REFATORACOES INDEPENDEDNTE DO !#_REFACTORINGTYPE
# Agrupamos pelo '10_HASH' e somamos as outras colunas
DF_REFACTORING = DF_REFACTORING.groupby(["10_HASH"]).sum().reset_index()

#crete a new column 'REFATORACOES' with default values as 1
DF_REFACTORING['REFATORACOES'] = 1

DF_REFACTORING.to_csv(FILE_NAME_REFACTORING_AGRUPADO, index=False)

In [35]:
#JUNTA O DF E O DF_REFACTORING, SE NAO TIVER VALOR EM ALGUMA COLUNA, PREENCHE COM 0
tabela_completa = pd.merge(df, DF_REFACTORING, how='outer', on='10_HASH')
tabela_completa.fillna(0, inplace=True)

tabela_completa.to_csv(FILE_NAME_TABELA_COMPLETA, index=False)
tabela_completa.drop(columns=['10_HASH', '11_PARENTSHASH'], inplace=True)

In [36]:
def transforma_valor(valor):
    if int(valor) > 0:
        return 1
    else:
        return int(valor)

# Aplica a função em cada elemento do DataFrame
tabela_completa = tabela_completa.applymap(transforma_valor)

C:\Users\kleit\AppData\Local\Temp\ipykernel_15660\1157776071.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tabela_completa = tabela_completa.applymap(transforma_valor)


In [37]:
tabela_completa

,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve,DIFERENCA_QTN_ISLINECOMMENT_Aumentou,DIFERENCA_QTN_ISLINECOMMENT_Diminuiu,DIFERENCA_QTN_ISLINECOMMENT_Manteve,DIFERENCA_QNT_SEGMENTOS_Aumentou,DIFERENCA_QNT_SEGMENTOS_Diminuiu,DIFERENCA_QNT_SEGMENTOS_Manteve,DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou,DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu,DIFERENCA_QTN_ISJAVADOCCOMMENT_Manteve,REFATORACOES
0,0,0,1,0,0,1,0,0,1,0,0,1,0
1,0,0,1,0,0,1,0,0,1,0,0,1,0
2,0,0,1,0,0,1,0,0,1,0,0,1,0
3,0,0,1,1,0,0,1,0,0,1,0,0,1
4,0,0,1,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,0,0,0,0,0,0,0,0,0,0,0,0,1
1834,0,0,0,0,0,0,0,0,0,0,0,0,1
1835,0,0,0,0,0,0,0,0,0,0,0,0,1
1836,0,0,0,0,0,0,0,0,0,0,0,0,1


In [38]:
itemsets = apriori(tabela_completa, min_support=0.01, use_colnames=True, max_len=2)


c:\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


# Regras

In [39]:

rules = association_rules(itemsets, metric="support", min_threshold=0.01)


In [40]:
# ordem by support
rules = rules.sort_values(by='support', ascending=False)
# rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
# rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))
rules.to_csv(FILE_NAME_TABELA_REGRAS, index=False)

In [41]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
25,(DIFERENCA_QTN_ISJAVADOCCOMMENT_Manteve),(DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve),0.791077,0.917301,0.785637,0.993122,1.082657,0.059980,12.024374,0.365427
24,(DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve),(DIFERENCA_QTN_ISJAVADOCCOMMENT_Manteve),0.917301,0.791077,0.785637,0.856465,1.082657,0.059980,1.455553,0.923185
12,(DIFERENCA_QTN_ISLINECOMMENT_Manteve),(DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve),0.745375,0.917301,0.738847,0.991241,1.080605,0.055113,9.441422,0.292952
13,(DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve),(DIFERENCA_QTN_ISLINECOMMENT_Manteve),0.917301,0.745375,0.738847,0.805457,1.080605,0.055113,1.308833,0.901984
57,(DIFERENCA_QTN_ISJAVADOCCOMMENT_Manteve),(DIFERENCA_QTN_ISLINECOMMENT_Manteve),0.791077,0.745375,0.702394,0.887895,1.191206,0.112744,2.271314,0.768296
...,...,...,...,...,...,...,...,...,...,...
36,(DIFERENCA_QNT_SEGMENTOS_Aumentou),(DIFERENCA_QTN_ISLINECOMMENT_Diminuiu),0.217628,0.042982,0.013058,0.060000,1.395949,0.003704,1.018105,0.362541
7,(DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou),(REFATORACOES),0.019042,0.267682,0.012514,0.657143,2.454936,0.007416,2.135927,0.604162
6,(REFATORACOES),(DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou),0.267682,0.019042,0.012514,0.046748,2.454936,0.007416,1.029064,0.809290
43,(DIFERENCA_QTN_ISLINECOMMENT_Diminuiu),(DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu),0.042982,0.021219,0.010881,0.253165,11.931191,0.009969,1.310572,0.957334
